In [ ]:
import itertools as it
from collections import Counter, defaultdict
from functools import partial
from glob import glob
from pathlib import Path

import gfapy
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
pl.enable_string_cache()

# Functions

In [ ]:
def concat_glob(filename):
    return pl.concat([pl.scan_ipc(f) for f in glob(filename)], how="diagonal")

In [ ]:
def label_columns(cols, func=None):
    expr = None
    for col in cols:
        if expr is None:
            expr = pl.when(pl.col(col).is_not_null())
        else:
            expr = expr.when(pl.col(col).is_not_null())
        if func is not None:
            lit = func(col)
        else:
            lit = col
        expr = expr.then(pl.lit(lit))
    return expr

# 240610_pLIB476 vs. 240510_pLIB473-476

In [ ]:
arrow_filename = "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/"
df = concat_glob(arrow_filename)  # .collect()

# prepare_reads bugfix

In [ ]:
arrow_filename = "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/work/ea/83943aa59f16e2657d0d323ba47785/input/channel-93_merged.arrow"
df = concat_glob(arrow_filename)  # .collect()

In [ ]:
gfa_filename = "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/references/pLIB476jqs.gfa"
gfa = gfapy.Gfa.from_file(gfa_filename)

In [ ]:
len(df)

In [ ]:
%%time
df2 = processing.cut_cigar_df(
    df,
    gfa,
    path_column="path",
    cigar_column="cg",
    # sequence_column="read_seq",
    # phred_column="read_phred",
    query_start_column="query_start",
    query_end_column="query_end",
    query_length_column="query_length",
    path_start_column="path_start",
    path_end_column="path_end",
    struct_name="extract_segments",
    keep_full=True,
    cut_cigar_kwargs=dict(
        key_sep="|",
        return_indices=False,
        return_counts=True,
        return_cigars=False,
        return_variants=True,
        separate_ends=True,
    ),
)

In [ ]:
%%time
df2 = df2.collect()

In [ ]:
df2

# Barcode alignment quality filtering

In [ ]:
arrow_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/prepare_reads/channel-100_merged.arrow"
# arrow_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/extract_segments/consensus-0-of-400.arrow"
df = concat_glob(arrow_filename).collect()

In [ ]:
gfa_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/references/barcode.gfa"
# gfa_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/references/pLIB430-435.gfa"
gfa = gfapy.Gfa.from_file(gfa_filename)

In [ ]:
len(df)

In [ ]:
%%time
df2 = processing.cut_cigar_df(
    df[:5000],
    gfa,
    path_column="full_path",
    cigar_column="cg",
    sequence_column="read_seq",
    phred_column="read_phred",
    query_start_column="query_start",
    query_end_column="query_end",
    query_length_column="query_length",
    path_start_column="path_start",
    path_end_column="path_end",
    keep_full=True,
    # CLI args: variant_sep, segments
    # cut_cigar_kwargs=dict(
    #     return_variants=False,
    #     return_cigars=False,
    #     return_counts=True,
    #     return_indices=False,
    # ),
)

In [ ]:
bit_segs = set([s.split("=")[0] for s in gfa.segment_names if s.startswith("BC:BIT")])

In [ ]:
df2.with_columns(
    **{
        f"{s}_div": pl.sum_horizontal(
            pl.col(f"{s}|mismatches"),
            pl.col(f"{s}|insertions"),
            pl.col(f"{s}|deletions"),
        )
        for s in bit_segs
    }
)

In [ ]:
df2.with_columns(
    max_divergence=pl.max_horizontal(
        [
            pl.sum_horizontal(
                pl.col(f"{s}|mismatches"),
                pl.col(f"{s}|insertions"),
                pl.col(f"{s}|deletions"),
            )
            / pl.col()
            for s in bit_segs
        ]
    )
)

In [ ]:
z = _["max_divergence"]

In [ ]:
plt.hist(z, log=True, cumulative=True, histtype="step", density=True, bins=50);

In [ ]:
df2.columns

In [ ]:
plt.hist(df["id"], cumulative=True, bins=100, density=True, histtype="step", log=True);

In [ ]:
plt.scatter(df["id"], df["NM"], s=0.3);

In [ ]:
plt.scatter(df["id"], df["NM"], s=0.3);

# cut_cigar optimization

In [ ]:
arrow_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/extract_segments/consensus-0-of-400.arrow"
# arrow_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/extract_segments/*.arrow"
df = concat_glob(arrow_filename).collect()

In [ ]:
# gfa_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/references/barcode.gfa"
gfa_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/references/pLIB430-435.gfa"
gfa = gfapy.Gfa.from_file(gfa_filename)
name_to_seq = sgfa.gfa_name_mapping(gfa)

In [ ]:
idx = 5
cigar = scigar.decode_cigar(df[idx, "cg"])
path = df[idx, "variants_path"].to_list()
seq = df[idx, "consensus_seq"]
path_start = df[idx, "path_start"]
path_end = df[idx, "path_end"]
query_start = df[idx, "query_start"]
query_end = df[idx, "query_end"]

In [ ]:
%%timeit
res = scigar.cut_cigar(
    cigar,
    path,
    name_to_seq,
    sequence=seq,
    path_start=path_start,
    path_end=path_end,
    query_start=query_start,
    query_end=query_end,
)

In [ ]:
res

In [ ]:
res

# Duplicate barcode debugging

In [ ]:
arrow_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/extract_segments/consensus-0-of-400.arrow"
# arrow_filename = "/home/jqs1/scratch/sequencing/231201_bcd_rbses_run3/20231201_1101_1F_PAU05823_773c75ee/extract_segments/*.arrow"
df = concat_glob(arrow_filename).collect()

In [ ]:
len(df)

In [ ]:
name_to_seq = sgfa.gfa_name_mapping(gfa)

In [ ]:
sum([len(name_to_seq[s]) for s in df["variants_path"][-3].to_list()])

In [ ]:
x

In [ ]:
def path_advance(c):
    l = 0
    for op, length in c:
        if op != scigar.CigarOp["D"]:
            l += length
    return l


def query_advance(c):
    l = 0
    for op, length in c:
        if op != scigar.CigarOp["I"]:
            l += length
    return l


def all_advance(c):
    l = 0
    for op, length in c:
        l += length
    return l


def advance(c):
    return (path_advance(c), query_advance(c), all_advance(c))


x = scigar.decode_cigar(df[1, "cg"])
advance(x)

In [ ]:
advance(a)

In [ ]:
advance(cigar2)

In [ ]:
len(df[1, "consensus_seq"])

In [ ]:
df[1]

In [ ]:
df["query_start"]

In [ ]:
df["query_length"]

In [ ]:
np.where(df["path_start"] != 0)

In [ ]:
(df["path_start"] != 0).sum()

In [ ]:
idx = 5

In [ ]:
df[idx]

In [ ]:
cigar = scigar.decode_cigar(df[idx, "cg"])
path = df[idx, "variants_path"].to_list()
a = scigar.cut_cigar(
    cigar,
    path,
    name_to_seq,
    sequence=df[idx, "consensus_seq"],
    path_start=df[idx, "path_start"],
    path_end=df[idx, "path_end"],
    query_start=df[idx, "query_start"],
    query_end=df[idx, "query_end"],
)

In [ ]:
a

In [ ]:
advance(a)

In [ ]:
a

In [ ]:
cigar

In [ ]:
advance(cigar)

In [ ]:
cigar2 = scigar.decode_cigar(df[idx, "realign_cg"])
path = df[idx, "variants_path"].to_list()
b = scigar.cut_cigar(cigar2, path, name_to_seq, sequence=df[idx, "consensus_seq"])

In [ ]:
b

In [ ]:
cigar2

In [ ]:
advance(cigar2)

In [ ]:
len(df[idx, "consensus_seq"])

In [ ]:
df[idx]

In [ ]:
a

In [ ]:
b

In [ ]:
len(name_to_seq["<UNS3"])

In [ ]:
cigar

In [ ]:
path

In [ ]:
a

In [ ]:
df.columns

In [ ]:
%%time
df2 = df.with_columns(
    dup=pl.col("name").is_duplicated(),
    e2e=pl.col("variants_path")
    .list.set_intersection(
        [
            "<BC:UPSTREAM",
            "<UNS3",
            ">BC:UPSTREAM",
            ">UNS3",
        ]
    )
    .list.len()
    == 2,
    bc_e2e=pl.col("variants_path")
    .list.set_intersection(
        [
            "<BC:UPSTREAM",
            "<BC:SPACER2",
            ">BC:UPSTREAM",
            ">BC:SPACER2",
        ]
    )
    .list.len()
    == 2,
)

In [ ]:
df3 = df2.filter(pl.col("e2e"), ~pl.col("dup"))

In [ ]:
len(df3)

In [ ]:
len(df3) / len(df2)

In [ ]:
df3.select(r"^.*\|insertions$")

In [ ]:
ins = df3.select(max=pl.max_horizontal(r"^.*\|insertions$"))["max"].sort()

In [ ]:
ins[-200]

In [ ]:
df3.group_by("grouping_path").agg(pl.len())["len"].value_counts()

In [ ]:
df.columns

In [ ]:
df.select(pl.exclude(r"^.*\|.*$"))

In [ ]:
df.group_by("grouping_path").agg(pl.col("name")).filter(
    pl.col("name").list.len() > 4
)  # ["len"].value_counts()

In [ ]:
df.filter(
    pl.col("name")
    == "consensus_4d03b99001bd2316ebc0c40067fa3dc8387e5fd61608dd45b4891ccf313ab769"
).select(pl.exclude(r"^.*\|.*$"))

In [ ]:
df.filter(
    pl.col("name")
    == "consensus_4d03b99001bd2316ebc0c40067fa3dc8387e5fd61608dd45b4891ccf313ab769"
).select(pl.exclude(r"^.*\|.*$"))[0, "cg"]

In [ ]:
df.filter(
    pl.col("name")
    == "consensus_4d03b99001bd2316ebc0c40067fa3dc8387e5fd61608dd45b4891ccf313ab769"
).select(pl.exclude(r"^.*\|.*$"))[0, "realign_cg"]

# parasail/pywfa local alignment

In [ ]:
res = align.pairwise_align("aaatctctctcgggg", "tctctctc", method="parasail")

In [ ]:
res

In [ ]:
res = align.pairwise_align("aaatctctctcgggg", "tctctctc", method="pywfa")

In [ ]:
res

In [ ]:
res = align.pairwise_align("tctctctc", "aaatctctctcgggg", method="parasail")

In [ ]:
res

In [ ]:
res

In [ ]:
res = align.pairwise_align("tctctctcgggg", "aaatctctctc", method="pywfa")

In [ ]:
res = align.pairwise_align("aaatctctctc", "tctctctcgggg", method="pywfa")

In [ ]:
res = align.pairwise_align("aaatctctc", "ggtctctcgggg", method="pywfa")

In [ ]:
res = align.pairwise_align("aaatctctc", "ggtctctcgggg", method="parasail")

In [ ]:
len("aaatctctc")

In [ ]:
len("aaatctctctcgggg")